In [ ]:
!pip install openai==0.28 requests python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.14.0
    Uninstalling openai-2.14.0:
      Successfully uninstalled openai-2.14.0


In [ ]:
# this code is for setting up weather API integration
from google.colab import userdata

import requests
import openai
#from config import WEATHER_API_KEY
#from config import OPENAI_API_KEY

WEATHER_API_KEY = userdata.get('WEATHER_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')


def get_weather(location):
    try:
        # base URL for the WeatherAPI current weather endpoint.
        url = f"http://api.weatherapi.com/v1/current.json"
        # creates a dictionary of parameters to be sent with the GET request
        params = {
            "key": WEATHER_API_KEY,
            "q": location,
            # specify that Air Quality Index data should not be included
            "aqi": "no"
        }
        # send an HTTP GET request to the WeatherAPI with the constructed URL and parameters
        response = requests.get(url, params=params)
        # raise exception for bad status codes
        # check if the request was successful. If the status code indicates an error
        # (e.g., 4xx or 5xx), it raises an HTTPError
        response.raise_for_status()
        # if the request is successful, it parses the JSON response from the API and returns it as a Python dictionary.
        return response.json()
    except requests.exceptions.RequestException as e:
        print("Exception occurred", e)


# test weather API integration
if __name__ == "__main__":
    try:
        weather_data = get_weather("Singapore")
        print(f"Location name: {weather_data['location']['name']}")
        print(f"Temperature in degree Celsius : {weather_data['current']['temp_c']}")
        print(f"Temperature in Fahrenheit: {weather_data['current']['temp_f']}")
        print(f"Condition: {weather_data['current']['condition']['text']}")
        print(f"Wind speed and direction: ", weather_data['current']["wind_mph"], weather_data['current']["wind_dir"])
        print(f"Humidity: ", weather_data['current']["humidity"])
        print(f"Feels like temperature in degree Celsius: ", weather_data['current']["feelslike_c"])
        print(f"Time of reading: ", weather_data['current']["last_updated"])
    except Exception as e:
        print("Exception occurred", e)


Location name: Singapore
Temperature in degree Celsius : 29.4
Temperature in Fahrenheit: 84.9
Condition: Partly cloudy
Wind speed and direction:  8.7 NE
Humidity:  66
Feels like temperature in degree Celsius:  34.9
Time of reading:  2026-01-15 19:30


In [ ]:
# add openAI function definition
openai.api_key = OPENAI_API_KEY

# define the function that openAI can call
functions = [
    {
        "name": "get_weather",
        "description": "function to fetch current weather information for a location",
        "parameters": {
            # indicates that the parameters are an object (like a dictionary).
            "type": "object",
            # describes the individual parameters
            "properties": {
                "location": {
                    "type": "string",
                    "description": "city name or location"
                }
            },
            # parameter names that are mandatory for this function call
            "required": ["location"]
        },
    }
]

# test OpenAI function definition
# block runs only when the script is executed directly
if __name__ == "__main__":
    try:
        # make a call to the OpenAI gpt-4 model
        response = openai.ChatCompletion.create(
            model="gpt-4",
            # provides a user query to the model
            messages=[{"role": "user", "content": "What's the weather like in Tokyo?"}],
            # informs the model about the get_weather() function defined above
            functions=functions,
            # tell the model to automatically decide whether to respond with text or by suggesting a function call
            function_call="auto"
        )
        print("OpenAI Response:")
        # extract and print the function call information returned by
        # OpenAI. For the query "What's the weather like in Tokyo?", the model
        # should correctly identify that it needs to call get_weather with location='Tokyo'.
        # print(response)
        # print("**************")
        print(response.choices[0].message.function_call)
    except Exception as e:
        print(f"Error: {e}")

OpenAI Response:
{
  "name": "get_weather",
  "arguments": "{\n  \"location\": \"Tokyo\"\n}"
}


In [ ]:
# implement the weather agent
# user_query is expected to be a string containing the user's question about the weather
def weather_agent(user_query):
    try:
        # Get function call from OpenAI
        response = openai.ChatCompletion.create(
            model="gpt-4",
            # This is a list of message dictionaries that represents the conversation
            # history. In this case, it contains a single message from the user
            # ("role": "user") with the content of the user_query.
            messages=[{"role": "user", "content": user_query}],
            # This parameter tells the OpenAI model about the available functions
            # (tools) it can use. The functions variable was defined earlier in
            #  the notebook and describes the get_weather function
            functions=functions,
            # instructs the model to automatically decide whether to respond with
            # a text message or by suggesting a function call based on the user's query and the available functions
            function_call="auto"
        )

        # Extract function call
        # After the OpenAI API call returns a response, this line extracts the details
        # of any function call that the model decided to make. response.choices[0]
        # gets the first (and usually only) choice from the model's response
        # .message accesses the message object, and .function_call specifically gets
        # the object containing the function's name and arguments if a function call was suggested.
        function_call = response.choices[0].message.function_call

        # Execute function
        if function_call.name == "get_weather":
            # Parse the arguments from JSON string
            import json
            # OpenAI model provides the arguments for the function call as a JSON string.
            # Below line uses json.loads() to parse that JSON string into a Python dictionary,
            #  making the arguments accessible (e.g., arguments['location']).
            arguments = json.loads(function_call.arguments)
            weather_data = get_weather(arguments["location"])

            # Format response
            return f"Current weather in {weather_data['location']['name']}: " \
                   f"{weather_data['current']['temp_c']}°C, " \
                   f"{weather_data['current']['condition']['text']}"
    except Exception as e:
        return f"Error: {str(e)}"


# Test the complete weather agent
if __name__ == "__main__":
    test_queries = [
        "What's the weather like in Singapore?",
        "Is it raining in London right now?",
        "Tell me the temperature in New York"
    ]

    for query in test_queries:
        print(f"\nQuery: {query}")
        print(f"Response: {weather_agent(query)}")


Query: What's the weather like in Singapore?
Response: Current weather in Singapore: 29.3°C, Partly cloudy

Query: Is it raining in London right now?
Response: Current weather in London: 9.0°C, Patchy light rain

Query: Tell me the temperature in New York
Response: Current weather in New York: 6.7°C, Overcast
